In [1]:
import json
import re
import requests

he_appid = 'KymxRBvsVraBqbxy6gCt'
he_appcode = 'LbNbmVAeiyg51Dbr2C_NOw'

lat = 1.29902
lng = 103.85878

def search_rtti(search_lat, search_lng):
    traffic_info = {}
    pde_api_url = 'https://pde.cit.api.here.com/1/search/proximity.json?layer_ids=LINK_FC1,LINK_FC2,LINK_FC3,LINK_FC4,LINK_FC5&proximity={},{},10&key_attributes=LINK_ID,LINK_ID,LINK_ID,LINK_ID,LINK_ID&app_id={}&app_code={}'.format(
        search_lat, search_lng, he_appid, he_appcode)
    print('Locate link at {},{}: '.format(search_lat, search_lng) + pde_api_url)
    pde_r = json.loads(requests.get(pde_api_url).text)
    pde_geometries = pde_r['geometries']
    if len(pde_geometries) > 0:
        for geometries_i in range(len(pde_geometries)):
            geometry = pde_geometries[geometries_i]['geometry']
            link_id = pde_geometries[geometries_i]['attributes']['LINK_ID']
            nums = re.findall(r'\d+(?:\.\d*)?', geometry)
            pde_lat_lng_list = []
            for i in range(int(len(nums) / 2)):
                pde_lat_lng_list.append([float(nums[i + 1]), float(nums[i])])
                nums.pop(0)
            print('Found LINK_ID:', link_id, '\nGeometry:', geometry)

        he_rtti_url = 'https://traffic.api.here.com/traffic/6.1/flow.json?prox={},{},10&responseattributes=sh&app_id={}&app_code={}'.format(
            search_lat, search_lng, he_appid, he_appcode)
        print('Request for RTTI : ', he_rtti_url)
        if len(requests.get(he_rtti_url).text) > 0:
            traffic_r = json.loads(requests.get(he_rtti_url).text)
            traffic_link_list = []
            if len(traffic_r['RWS']) > 0:
                created_timestamp = traffic_r['CREATED_TIMESTAMP']
                for rws_i in range(len(traffic_r['RWS'])):
                    rw = traffic_r['RWS'][rws_i]['RW']
                    table_id = traffic_r['RWS'][rws_i]['TABLE_ID']
                    for rw_i in range(len(rw)):
                        fis = rw[rw_i]['FIS']
                        li = rw[rw_i]['LI']
                        for fis_i in range(len(fis)):
                            fi = fis[fis_i]['FI']
                            for fi_i in range(len(fi)):
                                cf = fi[fi_i]['CF']
                                shp = fi[fi_i]['SHP']
                                tmc = fi[fi_i]['TMC']
                                for cf_i in range(len(cf)):
                                    ty = cf[cf_i]['TY']
                                    sp = cf[cf_i]['SP']
                                    su = cf[cf_i]['SU']
                                    ff = cf[cf_i]['FF']
                                    jf = cf[cf_i]['JF']
                                de = tmc['DE']
                                le = tmc['LE']
                                pc = tmc['PC']
                                qd = tmc['QD']
                                for shp_i in range(len(shp)):
                                    traffic_shape_point_list = []
                                    shape_values = shp[shp_i]['value']
                                    for shape_value in shape_values:
                                        shape_value_list = shape_value.split(' ')
                                        for shape_point in shape_value_list:
                                            if len(shape_point) > 0:
                                                shape_point_lat = float(shape_point.split(',')[0])
                                                shape_point_lng = float(shape_point.split(',')[1])
                                                traffic_shape_point_list.append([shape_point_lat, shape_point_lng])
                                    link_found = False
                                    for pde_shape_point in pde_lat_lng_list:
                                        if pde_shape_point not in traffic_shape_point_list:
                                            link_found = False
                                            break
                                        else:
                                            link_found = True
                                    if link_found:
                                        traffic_info = {'created_timestamp': created_timestamp,
                                                        'shape': traffic_shape_point_list, 'ty': ty, 'sp': sp,
                                                        'su': su, 'ff': ff, 'jf': jf, 'de': de, 'li': li, 'le': le,
                                                        'pc': pc, 'qd': qd}


        if len(traffic_info) > 0:
            print('Successfully mapping RTTI to LINK_ID={}'.format(link_id))
            print('RTTI: ', traffic_info)
        else:
            print('Failed to mapping RTTI.'.format(link_id))
    return traffic_info


In [2]:
import folium

lat = 1.29902
lng = 103.85878

rtti = search_rtti(lat, lng)

m = folium.Map(location=[lat, lng],
               tiles='https://1.traffic.maps.api.here.com/maptile/2.1/traffictile/newest/normal.traffic.day/{z}/{x}/{y}/256/png?app_id=KymxRBvsVraBqbxy6gCt&app_code=LbNbmVAeiyg51Dbr2C_NOw',
               attr='HERE')
folium.map.Marker([lat, lng]).add_to(m)
p = folium.vector_layers.PolyLine(rtti['shape'], popup=str(rtti))
b = p.get_bounds()
p.add_to(m)
m.fit_bounds(b)
m


Locate link at 1.29902,103.85878: https://pde.cit.api.here.com/1/search/proximity.json?layer_ids=LINK_FC1,LINK_FC2,LINK_FC3,LINK_FC4,LINK_FC5&proximity=1.29902,103.85878,10&key_attributes=LINK_ID,LINK_ID,LINK_ID,LINK_ID,LINK_ID&app_id=KymxRBvsVraBqbxy6gCt&app_code=LbNbmVAeiyg51Dbr2C_NOw
Found LINK_ID: 843979395 
Geometry: MULTILINESTRING((103.85869 1.29896,103.85893 1.29921))
Request for RTTI :  https://traffic.api.here.com/traffic/6.1/flow.json?prox=1.29902,103.85878,10&responseattributes=sh&app_id=KymxRBvsVraBqbxy6gCt&app_code=LbNbmVAeiyg51Dbr2C_NOw
Successfully mapping RTTI to LINK_ID=843979395
RTTI:  {'created_timestamp': '2019-03-25T04:42:02.000+0000', 'shape': [[1.29921, 103.85893], [1.29896, 103.85869]], 'ty': 'TR', 'sp': 25.36, 'su': 25.36, 'ff': 35.3, 'jf': 1.74724, 'de': 'Rochor Rd', 'li': 'A10-00192', 'le': 0.22907, 'pc': 196, 'qd': '+'}
